In [ ]:
%config IPCompleter.greedy=True

In [ ]:
from requests import get
from bs4 import BeautifulSoup
import pymongo
import datetime
import numpy as np
import re
from time import sleep

In [ ]:
rawHTML = get('https://www.infobae.com/politica')
rawHTML = rawHTML.text.split('pb-container')
html = BeautifulSoup(rawHTML[3], 'html.parser')

In [ ]:
allNews = html.find_all('a', href=True)#(attrs={'class':'col-sm-8'})

cursorNews = []

for link in allNews:
    if ('https://www.infobae.com' + link['href']) not in cursorNews and link['href'] != 'https://www.infobae.com/ultimas-noticias/':
        cursorNews.append('https://www.infobae.com' + link['href'])

In [ ]:
myclient = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = myclient["DiarioStage"]
mycol = mydb.Infobae

currentDate = datetime.datetime.now().date()
mongoCurrentDate = datetime.datetime(currentDate.year, currentDate.month, currentDate.day,0,0,0)

select = mycol.find({"date": {"$gte": mongoCurrentDate}})

In [ ]:
cursorNews

In [ ]:
for news in cursorNews:
    try:
        rawHTML = get(news)
        html = BeautifulSoup(rawHTML.content, 'html.parser')
        title = html.find('title').text
        shorttext = html.find(attrs={'class':'subheadline'}).text
        date = html.find(attrs={'class':'byline-date'}).text
        fulltext = ""
        tags = []


        p_all = set(html.find_all('p'))
        p_to_remove = set(html.find_all('p', attrs={"class":""}))

        p_needed = p_all - p_to_remove

        for p in p_needed:
            fulltext = fulltext + p.text

        for story_tag in html.find_all(class_='story-tag'):
            tags.append(story_tag.text)

        dayRange = re.search("\d", date)
        year = date[-5:]

        if "Enero" or "enero" in date: month = "01"
        if "Febrero" or "febrero" in date: month = "02"
        if "Marzo" or "marzo" in date: month = "03"
        if "Abril" or "abril" in date: month = "04"
        if "Mayo" or "mayo" in date: month = "05"
        if "Junio" or "junio" in date: month = "06"
        if "Julio" or "julio" in date: month = "07"
        if "Agosto" or "agosto" in date: month = "08"
        if "Septiembre" or "septiembre" in date: month = "09"
        if "Octubre" or "octubre" in date: month = "10"
        if "Noviembre" or "noviembre" in date: month = "11"
        if "Diciembre" or "diciembre" in date: month = "12"

        correctedDate = datetime.datetime(int(year), int(month), int(date[dayRange.start() : dayRange.end() + 1]),3,0,0)

        mykey = {"title": title}
        mydict = {"fulltext": fulltext, "shorttext": shorttext, "tags": tags, "date": correctedDate}

        x = mycol.update_one(mykey, {'$set' : mydict}, upsert=True)

        sleep(1)
    except:
        print(news)